In [ ]:
!nvidia-smi

Sun Aug 14 05:34:01 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P8    12W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
#drive.mount('/content/gdrive', force_remount=True)
drive.mount('/content/gdrive')
!ls
%cd gdrive/My\ Drive/MalMem/
!ls

Mounted at /content/gdrive
gdrive	sample_data
/content/gdrive/My Drive/MalMem
 adasyn_tl_xgboost_balance.ipynb
 adasyn_tl_xgboost_balance_noRUS.ipynb
 adasyn_tl_xgboost_imbalanced.ipynb
 clf_readcsv.py
 CNN_MalMem_balance
 CNN_MalMem_imbalance
 createCNN_Balanced.ipynb
 createCNN_Balanced_noRUS.ipynb（副本）
 createCNN_Imbalanced.ipynb
'Dynamic Malware Analysis with Feature Engineering and Feature Learning.pdf'
 ensemble_major_imb.ipynb
 ensemble_major.ipynb
 Evaluate_PR_3Algorithms.ipynb
 Evaluate_PR_CNN.ipynb
 Evaluate_PR_Voting.ipynb
 evaluate_PR_xgboost.ipynb
 evaluation_MM.xlsx
 logs
 MalMem_Test_x.csv
 MalMem_Test_y.csv
 MalMem_Train_x.csv
 MalMem_Train_y.csv
 MalMem_x.csv
 MalMem_y.csv
'Malware Memory Analysis _ Datasets _ Canadian Institute for Cybersecurity _ UNB.mhtml'
 myXgb_adasyn_MalMem_imb.json
 myXgb_adasyn_MalMem.json
 Obfuscated-MalMem2022.csv
 Obfuscated-MalMem2022-excel.xlsx
'Obfuscated Malware Detection Using Deep Generative Model based on Global_Local Features - Scienc

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Wed Jun  8 16:54:44 2022

@author: Administrator
"""
# -*- coding: utf-8 -*-
"""
Created on Fri Oct  1 00:05:33 2021
create network
@author: Administrator
CNN neural network running
"""

import time
import sys
start = time.time()

#add tensorflow. before keras
from tensorflow import keras
from tensorflow.keras.models import Sequential  #序贯模型
from tensorflow.keras.layers import Dense    #全连接层
from tensorflow.keras.layers import Dropout  #随机失活层
from tensorflow.keras.layers import Flatten  #展平层，从卷积层到全连接层必须展平
from tensorflow.keras.layers import Conv1D   #卷积层
from tensorflow.keras.layers import MaxPooling1D  #最大值池化
import pandas as pd
from tensorflow.keras import backend as k
#from sklearn.cross_validation import train_test_split #随机划分为训练子集和测试子集
from sklearn.model_selection import train_test_split  #leon
#from keras.utils.vis_utils import plot_model #leon
from tensorflow.keras.utils import  plot_model  #leon
from tensorflow.keras.optimizers import SGD 
import matplotlib.pyplot as plt
import pydot #leon
from tensorflow.keras.callbacks import TensorBoard
from sklearn import metrics
import tensorflow as tf
import numpy as np
from collections import Counter

batch_size = 128  #一批训练样本128张图片

num_classes = 4  # 总共4个类别
epochs = 40   #一共迭代20轮 epochs = 20   
log_folder = 'logs'
#class_names = ['benign 0', 'dos 1', 'probe 2', 'r2l 3', 'u2r 4']
#class_names = ['normal 0', 'dos 1', 'ddos 2', 'injection 3', 'xss 4','password 5','scanning 6', 'mitm 7']
c

# x_train = pd.read_csv('KDDTrain_x.csv',header=None).values
# y_train = pd.read_csv('KDDTrain_y.csv',header=None).values

#x_train = pd.read_csv('KDDTrain_x_20.csv',header=None)
#y_train = pd.read_csv('KDDTrain_y_20.csv',header=None)

x_train = pd.read_csv('MalMem_x.csv',header=None)
y_train = pd.read_csv('MalMem_y.csv',header=None)
train_x, test_x, train_Y, test_Y = train_test_split(x_train, y_train, test_size=0.3, random_state=42)

"""
train_x = pd.read_csv('MalMem_Train_x.csv',header=None)
train_Y = pd.read_csv('MalMem_Train_y.csv',header=None)
test_x = pd.read_csv('MalMem_Test_x.csv',header=None)
test_Y = pd.read_csv('MalMem_Test_y.csv',header=None)
"""

#y_train[0]=list(y_train[0].map({'benign':0, 'dos':1, 'probe':2, 'r2l':3, 'u2r':4}))
print("train_x", train_x.head())
print("train_Y", train_Y.head())

#y_train = y_train.values.ravel()
train_Y = train_Y.values.ravel()
test_Y  = test_Y.values.ravel()
# sys.exit(0)

print("before oversampling \n", pd.Series(train_Y).value_counts())
# td_train = pd.read_csv('KDDTest_x.csv',header=None)
# td_target = pd.read_csv('KDDTest_y.csv',header=None)

# x_test = pd.read_csv('KDDTest_x.csv',header=None).values
# y_test = pd.read_csv('KDDTest_y.csv',header=None).values
#x_test = pd.read_csv('KDDTest_x_20.csv',header=None)
#y_test = pd.read_csv('KDDTest_y_20.csv',header=None)
#y_test[0]=list(y_test[0].map({'benign':0, 'dos':1, 'probe':2, 'r2l':3, 'u2r':4}))
#print("x_test", x_test.head())
#print("y_test", y_test.head())
#y_test = y_test.values.ravel()
# sys.exit(0)

######--------------- RandomUnderSampler-------------------------
mean_class_size = int(pd.Series(train_Y).value_counts().sum()/4)
print("mean_class_size", mean_class_size)
ratio= {0: mean_class_size
        #'dos': mean_class_size,
        #'probe': mean_class_size,
        #'r2l': mean_class_size,
        #'u2r': mean_class_size
        }
# check if ratio para is still used
# print("before \n", pd.Series(train_Y).value_counts())
print('Original dataset shape %s' % Counter(train_Y))

from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(sampling_strategy=ratio, random_state=42)
train_x, train_Y = rus.fit_resample(train_x, train_Y)
print('now dataset shape %s' % Counter(train_Y))

#*******try using ADASYN on train dataset to make balance
#only 
from imblearn.over_sampling import ADASYN
ad = ADASYN (sampling_strategy='not majority', random_state=0)
# train_x_sm, train_Y_sm = sm.fit_resample(train_x, train_Y)
train_x_ad, train_Y_ad = ad.fit_resample(train_x, train_Y)
print('adasyn dataset shape %s' % Counter(train_Y_ad))

#Tomek Links
from imblearn.under_sampling import TomekLinks
sampling_strategy = "all"
tl = TomekLinks(sampling_strategy=sampling_strategy)
# x_res, Y_res = tl.fit_resample(train_x_sm, train_Y_sm)
x_res, Y_res = tl.fit_resample(train_x_ad, train_Y_ad)
print('Tomek Links dataset shape %s' % Counter(Y_res))
#*************************


#从训练集中手动指定验证集
# x_train, x_dev, y_train, y_dev = train_test_split(x_train, y_train, test_size=0.2, random_state=2)
#x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.15, random_state=2)
#input_shape = (41, 1)

# print("shape", x_train.shape[0])
# print("shape", x_dev.shape[0])

# create confusion_matrix

In [ ]:
import itertools 
import io
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import Adam
#from tensorflow.keras.optimizers import SGD

def plot_confusion_matrix(cm, class_names): 
    figure = plt.figure(figsize=(8, 8)) 
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Accent) 
    plt.title("Confusion matrix") 
    plt.colorbar() 
    tick_marks = np.arange(len(class_names)) 
    plt.xticks(tick_marks, class_names, rotation=45) 
    plt.yticks(tick_marks, class_names)

    cm = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)  
    threshold = cm.max() / 2. 

    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):   
        color = "white" if cm[i, j] > threshold else "black"   
        plt.text(j, i, cm[i, j], horizontalalignment="center", color=color)  
    
    plt.tight_layout() 
    plt.ylabel('True label') 
    plt.xlabel('Predicted label') 

    return figure

def plot_to_image(figure):    
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    plt.close(figure)
    buf.seek(0)

    digit = tf.image.decode_png(buf.getvalue(), channels=4)
    digit = tf.expand_dims(digit, 0)

    return digit

def log_confusion_matrix(epoch, logs):
    predictions = model.predict(test_x)
    predictions = np.argmax(predictions, axis=1)

    cm = metrics.confusion_matrix(test_Y, predictions)
    figure = plot_confusion_matrix(cm, class_names=class_names)
    cm_image = plot_to_image(figure)
    
    with file_writer_cm.as_default():
        tf.summary.image("Confusion Matrix", cm_image, step=epoch)

file_writer_cm = tf.summary.create_file_writer(log_folder)
#**********************************


if k.image_data_format() == 'channels_first':

   x_train_p = x_res.values.reshape(x_res.shape[0], 1, 50)
   test_x = test_x.values.reshape(test_x.shape[0], 1, 50)
   input_shape = (1, 50)
else:

   x_train_p = x_res.values.reshape(x_res.shape[0], 50, 1)
   test_x = test_x.values.reshape(test_x.shape[0], 50, 1)
   input_shape = (50, 1)
  
print(input_shape)
#x_train.head(5)


model = Sequential()  #sequential序贯模型:多个网络层的线性堆叠
#输出的维度（卷积滤波器的数量）filters=32；1D卷积窗口的长度kernel_size=3；激活函数activation   模型第一层需指定input_shape：
model.add(Conv1D(32, 3, activation='relu',input_shape=input_shape))  #data_format默认channels_last
model.add(MaxPooling1D(pool_size=(2))) #池化层：最大池化  池化窗口大小pool_size=2
model.add(Flatten())  #展平一个张量，返回一个调整为1D的张量
#model.add(Dropout(0.25))  #需要丢弃的输入比例=0.25    dropout正则化-减少过拟合  影响不大
model.add(Dense(128, activation='relu',name='fully_connected')) #全连接层
model.add(Dense(num_classes, activation='softmax',name='softmax'))

#编译，损失函数:多类对数损失，用于多分类问题， 优化函数：adadelta， 模型性能评估是准确率
#binary classification
#model.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])
#model.compile(loss='sparse_categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])
#multiclassification
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy']) 
#model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(lr=0.0005), metrics=['accuracy']) 

#model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['sparse_categorical_accuracy']) 
#运行 ， verbose=1输出进度条记录 0-不输出      epochs训练的轮数     batch_size:指定进行梯度下降时每个batch包含的样本数
#model.fit(x_train, y_train, batch_size= batch_size, epochs=epochs, verbose=1)

model.optimizer.get_config()

(50, 1)


{'amsgrad': False,
 'beta_1': 0.9,
 'beta_2': 0.999,
 'decay': 0.0,
 'epsilon': 1e-07,
 'learning_rate': 0.001,
 'name': 'Adam'}

In [ ]:



%reload_ext tensorboard

callbacks = [
   TensorBoard(log_dir=log_folder, 
               histogram_freq=1, 
               write_graph=True,
               write_images=True,
               update_freq='epoch',
               profile_batch=2,
               embeddings_freq=1),
   keras.callbacks.LambdaCallback(on_epoch_end=log_confusion_matrix)
]

"""
callbacks = [TensorBoard(log_dir=log_folder,
                         histogram_freq=1,
                         write_graph=True,
                         write_images=True,
                         update_freq='epoch',
                         profile_batch=2,
                         embeddings_freq=1)]
"""
history = model.fit(x_train_p, Y_res, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(test_x, test_Y),\
                    callbacks=callbacks)


#原程序也注释的部分
#模型的训练损失（loss）和验证损失（val_loss），以及训练准确率（acc）和验证准确率（val_acc）可以使用绘图代码绘制出来
def smooth_curve(points,factor=0.8): #定义使曲线变得平滑
    smoothed_points = []
    for point in points:
        if smoothed_points:
            previous = smoothed_points[-1]
            smoothed_points.append(previous * factor + point * (1 - factor))
        else:
            smoothed_points.append(point)
    return smoothed_points

loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs,loss, 'bo', label = 'Training loss')
plt.plot(epochs,val_loss, 'b', label = 'Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.figure()

acc = history.history['accuracy'] 
#acc = history.history['sparse_categorical_accuracy'] 

val_acc = history.history['val_accuracy'] 
#val_acc = history.history['val_sparse_categorical_accuracy'] 

plt.plot(epochs, acc,'bo', label = 'Training acc')
plt.plot(epochs, val_acc, 'b', label = 'Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.show()


# 将测试集输入到训练好的模型中，查看测试集的误差
#score = model.evaluate(x_test, y_test, verbose=0,batch_size= batch_size)
score = model.evaluate(test_x, test_Y, verbose=0, batch_size= batch_size)
print('Validation loss:', score[0])
print('Validation accuracy: %.6f%%' % (score[1] * 100))

#model.save('CNN-test-data-balanced-dropout')  #dropout
model.save('CNN_MalMem_balance')
#results = confusion_matrix(test_Y, pred_Y, labels= clf.classes_)
#print results


# use testdata to test the model
# score2 = model.evaluate(td_train, td_target, verbose=0, batch_size= batch_size)
# print('Testdata loss:', score2[0])
# print('Testdata accuracy: %.2f%%' % (score2[1] * 100))
%tensorboard --logdir={log_folder}

#运行的时间
stop = time.time()
print(str(stop-start) + "秒")

# 神经网络可视化
#plot_model(model, to_file='D:/model-nslkdd.png',show_shapes=True)

#输出模型各层参数情况
model.summary()

In [ ]:
from imblearn.metrics import geometric_mean_score, classification_report_imbalanced
from sklearn.metrics import confusion_matrix, zero_one_loss, accuracy_score, \
    classification_report, roc_auc_score, roc_curve, auc, plot_confusion_matrix, ConfusionMatrixDisplay

class_names_num = ['0', '1', '2', '3']
predictions = model.predict(test_x)
print("predictions shape:", predictions.shape)
print(predictions)

classes = np.argmax(predictions, axis = 1)
print("classes:", classes)

print(classification_report(test_Y, classes, target_names=class_names))   #Y_res
print(classification_report_imbalanced(test_Y, classes, digits=4, target_names=class_names))  #Y_res
print(f"The geometric mean is {geometric_mean_score(test_Y, classes):.3f}") #Y_res


#y_test = y_test.values.ravel()
#cm = confusion_matrix(y_test, classes, labels=class_names_num)
cm = confusion_matrix(test_Y, classes)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names_num)
disp.plot()
_ = disp.ax_.set_title("CNN")
plt.show()

In [ ]:
rac = roc_auc_score(test_Y, predictions, multi_class='ovr', average ='macro')  #Y_res
print("ROC AUC Score:", rac)

ROC AUC Score: 0.9423714027407903


In [ ]:
# plot roc curve
from itertools import cycle
import matplotlib.pyplot as plt
from sklearn.preprocessing import label_binarize
from sklearn.metrics import confusion_matrix, zero_one_loss, accuracy_score, \
    classification_report, roc_auc_score, roc_curve, auc, plot_confusion_matrix, ConfusionMatrixDisplay

y = label_binarize(test_Y, classes=[0, 1, 2, 3])  #Y_res
n_classes = y.shape[1]
predictions = model.predict(test_x)
yy = predictions

print("y:", y)
print("n_classes:", n_classes)

lw = 3

fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    # fpr[i], tpr[i], _ = roc_curve(Y_res[:, i], y[:, i])
    # fpr[i], tpr[i], threshold = roc_curve(y[:, i], yy[:, i], pos_label=0)
    fpr[i], tpr[i], threshold = roc_curve(y[:, i], yy[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(y.ravel(), yy.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])


# First aggregate all false positive rates
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

# Then interpolate all ROC curves at this points
mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])

# Finally average it and compute AUC
mean_tpr /= n_classes

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

# Plot all ROC curves
plt.figure()
plt.plot(
    fpr["micro"],
    tpr["micro"],
    label="micro-average ROC curve (area = {0:0.2f})".format(roc_auc["micro"]),
    color="deeppink",
    linestyle=":",
    linewidth=4,
)


plt.plot(
    fpr["macro"],
    tpr["macro"],
    label="macro-average ROC curve (area = {0:0.2f})".format(roc_auc["macro"]),
    color="navy",
    linestyle=":",
    linewidth=4,
)



colors = cycle(["aqua", "darkorange", "cornflowerblue", "#FF6666"]) #, "green", "yellow", "slategray", "blueviolet"])
for i, color in zip(range(n_classes), colors):
    plt.plot(
        fpr[i],
        tpr[i],
        color=color,
        lw=lw,
        label="ROC curve of class {0} (area = {1:0.2f})".format(i, roc_auc[i]),
    )

plt.plot([0, 1], [0, 1], "k--", lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Receiver operating characteristic of CNN multiclass")
plt.legend(loc="lower right")
plt.show()


# compute roc_auc_score
# y_prob = classifier.predict_proba(X_test)

macro_roc_auc_ovo = roc_auc_score(test_Y, yy, multi_class="ovo", average="macro")  #Y_res
weighted_roc_auc_ovo = roc_auc_score(
    test_Y, yy, multi_class="ovo", average="weighted"
)
macro_roc_auc_ovr = roc_auc_score(test_Y, yy, multi_class="ovr", average="macro")
weighted_roc_auc_ovr = roc_auc_score(
    test_Y, yy, multi_class="ovr", average="weighted"
) #Y_res
print(
    "One-vs-One ROC AUC scores:\n{:.6f} (macro),\n{:.6f} "
    "(weighted by prevalence)".format(macro_roc_auc_ovo, weighted_roc_auc_ovo)
)
print(
    "One-vs-Rest ROC AUC scores:\n{:.6f} (macro),\n{:.6f} "
    "(weighted by prevalence),\n{:.6f}"   #leon
    "(micro)".format(macro_roc_auc_ovr, weighted_roc_auc_ovr, roc_auc["micro"])  #leon
    #"(weighted by prevalence)".format(macro_roc_auc_ovr, weighted_roc_auc_ovr)
)